# Credit Card Fraud Detection

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../dataset/creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
data.sample(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
62873,50461.0,-0.929423,-0.230850,2.239642,-0.622143,-0.747027,0.693916,0.297491,0.360018,0.129358,...,0.013037,-0.032038,0.267946,0.112356,-0.353984,1.275938,-0.021873,0.092166,152.13,0
73333,55087.0,1.146695,0.031921,-0.186913,0.956262,0.253114,0.280441,0.070548,0.153111,0.092139,...,0.003550,0.047815,-0.228849,-0.672101,0.780425,-0.221071,0.008256,-0.004282,33.92,0
157128,109559.0,-0.735219,0.980973,2.188423,-0.372297,0.343890,0.239918,0.432699,-0.095709,1.408911,...,-0.303379,-0.347797,-0.354294,-0.491505,0.326281,-0.718432,0.029372,-0.094963,2.30,0
229233,145848.0,2.137061,0.103620,-2.486617,0.096313,0.807454,-1.394665,0.868925,-0.504654,-0.286295,...,0.280411,0.951819,-0.214320,-0.228607,0.616693,1.020206,-0.153076,-0.112088,2.00,0
91137,63328.0,1.098240,0.233536,0.565405,1.226902,-0.501793,-1.033403,0.262437,-0.281101,-0.129224,...,0.014491,-0.015116,-0.042621,0.736630,0.511948,-0.468859,0.018516,0.041338,62.48,0
61401,49815.0,-1.632933,0.059875,1.759088,-0.255133,0.734990,-0.452811,0.437363,0.039810,-0.277958,...,-0.125243,-0.407841,-0.240102,0.029989,0.501780,0.173050,-0.373031,0.320057,38.42,0
230062,146170.0,-0.476020,1.156489,1.967097,3.930087,0.910186,2.193240,-0.047045,0.532471,-2.117788,...,0.225333,0.537294,-0.373719,-0.358269,0.268825,0.584390,0.097553,0.088350,34.05,0
243718,152043.0,2.209958,-1.268961,-1.540295,-1.839839,-0.140742,0.602370,-0.959306,0.041599,-1.754894,...,0.162922,1.084851,0.181636,-0.232374,-0.083799,0.130019,0.041645,-0.063079,15.00,0
284746,172735.0,-0.899559,-0.047744,0.717555,-3.262816,-0.384415,-0.164529,-0.227184,0.272726,-1.835745,...,-0.289810,-0.464731,-0.434248,0.009776,0.803066,-0.129513,-0.176856,-0.190917,10.00,0
103177,68535.0,-1.746279,1.115847,2.492159,2.619597,-0.433320,0.739785,-0.580122,0.677893,-0.784400,...,0.155398,0.952813,0.095794,0.010705,0.249511,0.416792,0.347509,0.009114,8.04,0


In [4]:
data["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [5]:
data.shape

(284807, 31)

In [6]:
# checking for missing values
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
# exploratory data analysis (eda)